In [2]:
%load assignment_3.py
%run assignment_3.py

"""
# D7055E - Assigment 2
## Group 6
*   Donato Scarano - DONSCA-3
*   Jacob Yousif - JACYOU-0
*   Yuehua Qin - YUEQIN-9
"""

import numpy as np
import math
import cv2
import scipy.signal as signal
import matplotlib.pyplot as plt





"""Image Capture"""
cap = cv2.VideoCapture(0) 
if not cap.isOpened():
    print("Error: Camera not accessible")
    exit()

ret, frame = cap.read()

if not ret:
    print("Error: Failed to capture frame")
    cap.release()
    exit()

if frame is None or frame.size == 0:
    print("Error: frame is empty or not loaded correctly")
else:
    img = cv2.flip(frame, 1)
    img = cv2.flip(img, 0)
    cv2.imshow('Video Capture', img)
    cv2.waitKey(0)

cap.release()
cv2.destroyAllWindows()




"""Video Capture"""
fileName='output_detection.avi'
imgSize=(640,480)
frame_per_second=30.0
writer = cv2.VideoWriter(fileName, cv2.VideoWriter_fourcc(*"MJPG"), frame_per_second, imgSize)

cap = cv2.VideoCapture(0)
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        writer.write(frame)
        cv2.imshow('Video Capture',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
writer.release()
cv2.destroyAllWindows()



"""Load & Paly The Captured Video"""
fileName='output.avi' 
cap = cv2.VideoCapture(fileName)
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()



"""Change Window Size"""
scaling_factorx=0.5
scaling_factory=0.5
cap = cv2.VideoCapture(0)  
while(True):
    ret, frame = cap.read()
    frame=cv2.resize(frame,None,fx=scaling_factorx,fy=scaling_factory,interpolation=cv2.INTER_AREA)
    img = frame
    cv2.imshow('Smaller Window',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



"""Gray"""
cap = cv2.VideoCapture(0)
while(True):
    ret, frame = cap.read()
    img = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    cv2.imshow('Gray', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):  
        break

cap.release()
cv2.destroyAllWindows()


"""'Histogram Equalization'"""
def equalizeHistColor(frame):
    img = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
    img[:, :, 2] = cv2.equalizeHist(img[:, :, 2])  
    return cv2.cvtColor(img, cv2.COLOR_HSV2RGB)  

cap = cv2.VideoCapture(0)
while(True):
    ret, frame = cap.read()

    img = equalizeHistColor(frame)
    cv2.imshow('Histogram Equalization', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


"""'Wrap Filter'"""
def WarpImage(frame):
    ax,bx=10.0,100
    ay,by=20.0,120
    img_np=np.zeros(frame.shape,dtype=frame.dtype)
    rows,cols=img_np.shape[:2]

    for i in range(rows):
        for j in range(cols):
            offset_x=int(ax*math.sin(2*math.pi*i/bx))
            offset_y=int(ay*math.cos(2*math.pi*j/by))
            if i+offset_y<rows and j+offset_x<cols:
                img_np[i,j]=frame[(i+offset_y)%rows,(j+offset_x)%cols]
            else:
                img_np[i,j]=0
    return img_np

def equalizeHistColor(frame):
    img = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
    img[:, :, 2] = cv2.equalizeHist(img[:, :, 2])  
    return cv2.cvtColor(img, cv2.COLOR_HSV2RGB)  

cap = cv2.VideoCapture(0)
while(cap.isOpened()):
    ret, frame = cap.read()
    frame=cv2.resize(frame,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)

    if ret==1:
        img = equalizeHistColor(WarpImage(frame))
    else:
        img = equalizeHistColor(frame)

    cv2.imshow('Warped', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):  
        break

cap.release()
cv2.destroyAllWindows()


"""'Optical flow'"""
cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()

prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

while(1):
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    prvs = next

    cv2.imshow('Optical Flow', bgr)
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



"""'Background_Video_File'"""
video_file = cv2.VideoCapture('Background_Video_File.avi')
background_subtractor = cv2.createBackgroundSubtractorMOG2(varThreshold=32, detectShadows=False)


"""'Frames'"""
fig, axes = plt.subplots(10, 2, figsize=(25, 100), constrained_layout=True)
for i in range(10):
    ret, frame = video_file.read()
    if not ret:
        break  
    mask = background_subtractor.apply(frame)
    ax = axes[i, 0]
    ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('Frame {}'.format(i+1), fontsize=15, pad=20)  
    ax = axes[i, 1]
    ax.imshow(mask, cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('Mask: Frame {}'.format(i+1), fontsize=15, pad=20)  

plt.show()


"""'Borderline detection'"""
kernel = np.ones((5,5), np.uint8)
closing = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=3)
dilation = cv2.dilate(closing, kernel, iterations=3)
contours, _ = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
largest_contour = contours[np.argmax([cv2.contourArea(c) for c in contours])]
x, y, w, h = cv2.boundingRect(largest_contour)
template = frame[y: y+h, x: x+w]
plt.figure(figsize=(10, 12))
plt.imshow(cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 255), 1))
plt.xlabel('Borderline detection box around the walking man', fontsize=20)
plt.xticks([])
plt.yticks([])
plt.show()



"""'Exhaustive search'"""
fig, axes = plt.subplots(10, 1, figsize=(15, 100), constrained_layout=True)
for i in range(10):
    ret, next_frame = video_file.read()
    if not ret:
        print("Failed to grab frame.")
        break  
    next_frame_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY).astype('float64')
    next_frame_gray -= next_frame_gray.mean()
    template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY).astype('float64')
    template_gray -= template_gray.mean()
    corr = signal.correlate2d(next_frame_gray, template_gray, mode='same', boundary='symm')
    y, x = np.unravel_index(np.argmax(corr), corr.shape)
    top_left = (x - w // 2, y - h // 2)
    bottom_right = (top_left[0] + w, top_left[1] + h)
    marked_frame = cv2.rectangle(next_frame.copy(), top_left, bottom_right, (255, 255, 255), 1)
    marked_frame = marked_frame.astype('uint8')
    ax = axes[i]
    ax.imshow(marked_frame)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel(f'Frame {i+1}: Borderline detection box around walking man', fontsize=12)

plt.show()
video_file.release()


Error: Failed to capture frame


NameError: name 'exit' is not defined

Error: Failed to capture frame
Error: frame is empty or not loaded correctly


error: OpenCV(4.7.0) D:\bld\libopencv_1675730129252\work\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
%run myfile.py